In [41]:
# load module
import xarray as xr
import numpy as np
from skimage.segmentation import flood_fill

In [42]:
# compute closest ocean coastal cell
def get_nearest_ocean_point_1d(xlon, ylat, lon1d, lat1d):
    """needle is a single (lat,long) tuple.
          haystack is a numpy array to find the point in
          that has the shortest distance to needle
    """
    earth_radius_miles = 6371.0
    dlat = lat1d[:] - ylat
    dlon = lon1d[:] - xlon
    a = np.square(np.sin(dlat/2.0)) + np.cos(ylat) * np.cos(lat1d[:]) * np.square(np.sin(dlon/2.0))
    [iptsmin] = np.nonzero(a==np.min(a))   # the minimun location can be retreive at this stage
    
    if len(iptsmin) > 1:
        great_circle_distance = 2 * np.arcsin(np.sqrt(a[iptsmin]))
        d = earth_radius_miles * great_circle_distance
        print('find more than 1 point, I don\'t know what to do,')
        print([iptsmin],lon1d[iptsmin],lat1d[iptsmin],mask1d[iptsmin],d)
        print(np.min(a[mask1d != 0]))
        print('I pickup the first one')
    elif len(iptsmin) == 0:
        print('find 0 point, I don\'t know what to do, exit 42')
        sys.exit(42)
    return iptsmin[0]

In [43]:
def get_trg_coastal_points(ds,yseed,xseed):
    
    ## get coastal points
    msk_oce=ds.tmask.values[0,0,:,:].squeeze()
    
    print('ni/nj out')
    nj_out=msk_oce.shape[0]
    ni_out=msk_oce.shape[1]
    print(ni_out, nj_out)
    
    mask_trg=np.zeros(shape=(nj_out,ni_out))
    zmask=np.zeros(shape=(nj_out+1,ni_out+2))
    zmask[1::,1:-1]=msk_oce
    # periodicity
    zmask[:, 0]=zmask[:,-2]
    zmask[:,-1]=zmask[:, 1]
    zmask[0,:]=0
    
    # filling
    zmask = flood_fill(zmask, (yseed, xseed), 2)
    zmask[zmask!=2]=1
    zmask[zmask==2]=0

    print('get coastal mask')
    # build coastal mask
    mask_trg[0:-1,:]= ( zmask[1:-1,1:-1] + 
                              zmask[0:-2,1:-1] + zmask[2::,1:-1] + zmask[1:-1,0:-2] + zmask[1:-1,2::] + 
                              zmask[0:-2,0:-2] + zmask[2::,2::]  + zmask[2::,0:-2]  + zmask[0:-2,2::] ) * zmask[1:-1,1:-1]
    mask_trg[(mask_trg > 1) & (mask_trg < 9)] = 10
    mask_trg[mask_trg != 10] = np.nan
    mask_trg=mask_trg.astype(np.float32)
    mask_trg[mask_trg==10]=1

    print('get 1d array of coastal points')
    iy_trg,ix_trg=np.where(mask_trg==1)
    mask_trg_1d=mask_trg[iy_trg,ix_trg]

    lon=np.radians(ds.nav_lon.values.squeeze())
    lat=np.radians(ds.nav_lat.values.squeeze())
    lon_1d=lon[iy_trg,ix_trg]
    lat_1d=lat[iy_trg,ix_trg]
    
    return iy_trg,ix_trg,mask_trg_1d,lon_1d,lat_1d

In [44]:

# get Ant NEMO coastal points
ds_trg=xr.open_dataset('mesh_mask.nc')

## Get Antarctica mask point
yant=10
xant=10
iy_trg,ix_trg,mask_coast_trg_1d,lon_trg_rad_1d,lat_trg_rad_1d=get_trg_coastal_points(ds_trg,yant,xant)

ni/nj out
180 148
get coastal mask
get 1d array of coastal points


In [45]:
# Get Ant IMBIE coastal points
# ## Input data
print('open input file')
cf_in='Mask_SubRegions_IMBIE.nc'
ds_in=xr.open_dataset(cf_in)

print('get coastal trg mask')
mask_in=np.where(ds_in.SubBasins.values[:,:] < 200, 1, 0)
mask_coast_in=np.zeros(shape=mask_in.shape)
mask_coast_in[1:-1,1:-1]= ( mask_in[1:-1,1:-1] + 
                            mask_in[0:-2,1:-1] + mask_in[2::,1:-1] + mask_in[1:-1,0:-2] + mask_in[1:-1,2::] +
                            mask_in[0:-2,0:-2] + mask_in[2::,2::]  + mask_in[2::,0:-2]  + mask_in[0:-2,2::] ) * mask_in[1:-1,1:-1]
mask_coast_in[(mask_coast_in > 1) & (mask_coast_in < 9)] = 10
mask_coast_in[mask_coast_in!=10]=np.nan
mask_coast_in=mask_coast_in.astype(np.float32)
mask_coast_in[mask_coast_in==10]=1

print('get BedMachine coastal input points')
ij_lst=np.where(mask_coast_in == 1)
print(len(ij_lst[0]))

def get_1d_data(ds,ptslst):
    print('get lon 1d')
    lon1d=np.radians(ds.lon.values[ptslst[0],ptslst[1]])
    print('get lat 1d')
    lat1d=np.radians(ds.lat.values[ptslst[0],ptslst[1]])
    print('get basin 1d')
    b1d=ds.SubBasins.values[ptslst[0],ptslst[1]]
    return lon1d,lat1d,b1d
lon_in_rad_1d,lat_in_rad_1d,bas_in_1d=get_1d_data(ds_in,ij_lst)

open input file
get coastal trg mask
get BedMachine coastal input points
20873
get lon 1d
get lat 1d
get basin 1d


In [46]:
# Fill Ant coastal NEMO points with id
# for each NEMO coastal point, pickup the IMBIE value the closest
basinid_trg=np.zeros(shape=(ds_trg.tmask.values[0,0,:,:].squeeze().shape))
print(len(iy_trg))
for ipt in range(0,len(iy_trg)):
    if ipt%100 == 0:
        print(ipt)
    # get closest point
    inearest=get_nearest_ocean_point_1d(lon_trg_rad_1d[ipt], lat_trg_rad_1d[ipt],
                                       lon_in_rad_1d, lat_in_rad_1d)
    basinid_trg[iy_trg[ipt],ix_trg[ipt]]=bas_in_1d[inearest]

294
0
100
200


In [47]:
# get 1d id array for correspondance
idlist=set(bas_in_1d)
idlist

{1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0}

In [48]:
# do the same for Greenland
## Get Greenland mask point
ygreen=132
xgreen=125
iy_trg,ix_trg,mask_coast_trg_1d,lon_trg_rad_1d,lat_trg_rad_1d=get_trg_coastal_points(ds_trg,ygreen,xgreen)

ni/nj out
180 148
get coastal mask
get 1d array of coastal points


In [49]:
# for each NEMO coastal point, pickup the IMBIE value the closest
print(iy_trg,ix_trg)
for ipt in range(0,len(iy_trg)):
    if ipt%100 == 0:
        print(ipt)
    # get closest point
    inearest=get_nearest_ocean_point_1d(lon_trg_rad_1d[ipt], lat_trg_rad_1d[ipt],
                                       lon_in_rad_1d, lat_in_rad_1d)
    basinid_trg[iy_trg[ipt],ix_trg[ipt]]=99
    
idlist.add(99)

[117 117 117 118 118 118 119 119 119 119 120 120 121 121 121 122 122 122
 122 123 123 123 124 124 124 124 124 125 125 125 126 126 126 127 127 127
 128 128 129 129 130 130 130 131 131 131 132 132 133 133 134 134 134 135
 135 135 135 135 136 136 137 137 138 138 138 139 139 139 140 140 140 140
 140 140 141 141 141 142 142 143 143 143 144 144 144 145 145 145 146 146
 146 146 146 146] [118 119 120 117 118 120 116 117 120 121 116 121 116 121 122 116 122 123
 124 116 124 125 116 125 126 127 128 116 128 129 116 129 130 116 117 130
 117 130 117 130 117 130 131 117 118 131 118 131 118 131 118 119 131 117
 118 119 131 132 117 132 117 132 117 131 132 117 130 131 117 126 127 128
 129 130 117 125 126 117 125 117 124 125 117 123 124 117 122 123 117 118
 119 120 121 122]
0


In [56]:
# for all other calving point:
ds_calv=xr.open_dataset('calving.nc')
ij_clv=np.where((ds_calv.calving[0,:,:]!= 0) & (basinid_trg[:,:] == 0))
basinid_trg[ij_clv[0],ij_clv[1]]=100
idlist.add(100)

In [57]:
# get 1d id array for correspondance
idlst=np.array(sorted(idlist)).astype(int)
idlst

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  99, 100])

In [58]:
# Write netcdf
# ## Write output
# define data with variable attributes
data_vars = {'basinid':(['y','x'], basinid_trg,
                          {'units': '[]', 
                           'long_name':'basinid'}),
             'basins':(['basins'],np.array(sorted(idlist)).astype(int),
                         {'unit': '[]'})
                          }

# define coordinates
coords = {'nav_lon': (['y','x'], ds_trg.nav_lon.values.squeeze()),
          'nav_lat': (['y','x'], ds_trg.nav_lat.values.squeeze())}

# define global attributes
attrs = {'source':cf_in, 
         'method':('basinid is the id of the closest cell within source file')
         }

# create dataset
ds_out = xr.Dataset(data_vars=data_vars, 
                coords=coords, 
                attrs=attrs)

ds_out.to_netcdf('toto.nc')